<a href="https://colab.research.google.com/github/narendra-mds/CS5660/blob/main/CS5660_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
import numpy as np
import tarfile
import os
from sklearn.preprocessing import OneHotEncoder
from scipy.special import log_softmax

In [2]:
current_directory = os.getcwd()

print("Current working directory:", current_directory)

Current working directory: /content


In [3]:
def load_fvecs_from_tar(tar_filename, fvecs_filename):
    with tarfile.open(tar_filename, 'r') as tar:
        # Extract the fvecs file from the tar archive
        fvecs_file = tar.extractfile(fvecs_filename)
        with fvecs_file as f:
          fv = np.frombuffer(f.read(), dtype=np.float32)
          if fv.size == 0:
            return np.zeros((0, 0))
        dim = fv.view(np.int32)[0]
        fv = fv.reshape(-1, 1 + dim)
        if not all(fv.view(np.int32)[:, 0] == dim):
          raise IOError("Non-uniform vector sizes in " + fvecs_file)
        fv = fv[:, 1:]
        fv = fv.copy()
    return fv

In [4]:
def load_ivecs_from_tar(tar_filename, ivecs_filename):
    with tarfile.open(tar_filename, 'r') as tar:
        # Extract the ivecs file from the tar archive
        ivecs_file = tar.extractfile(ivecs_filename)
        with ivecs_file as f:
          a = np.frombuffer(f.read(), dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().reshape(-1)

## Read the Images and Labels

Note that the compressed .tgz file is uploaded to session data i.e. in \content

In [5]:
tar_file = 'groupFungus_k64_nclass10_nex10.tgz'
folder = 'example_data'
fungus10_train_images = os.path.join(folder, 'groupFungus_k64_nclass10_nex10_Xtrain.fvecs')
fungus10_train_labels = os.path.join(folder, 'groupFungus_k64_nclass10_nex10_Ltrain.ivecs')

In [6]:
fungus10_test_images = os.path.join(folder, 'groupFungus_k64_nclass10_nex10_Xtest.fvecs')
fungus10_test_labels = os.path.join(folder, 'groupFungus_k64_nclass10_nex10_Ltest.ivecs')

In [7]:
fungus10_train_images_features = load_fvecs_from_tar(tar_file, fungus10_train_images)
fungus10_test_images_features = load_fvecs_from_tar(tar_file, fungus10_test_images)

In [8]:
fungus10_train_images_features.shape, fungus10_test_images_features.shape

((100, 4096), (100, 4096))

In [9]:
fungus10_train_images_labels = load_ivecs_from_tar(tar_file, fungus10_train_labels)
fungus10_test_images_labels = load_ivecs_from_tar(tar_file, fungus10_test_labels)

In [10]:
fungus10_train_images_labels.shape, fungus10_test_images_labels.shape

((100,), (100,))

## Build a Linear model as described in the paper

First create a one hot encoded array for true values

In [21]:
# define one hot encoding
encoder = OneHotEncoder(sparse_output=False)
# transform data
y_train_true = encoder.fit_transform(fungus10_train_images_labels.reshape(100,1))
print(y_train_true[0])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Now build X matrix and Weight matrix based on dimensions of the data

In [32]:
X_train = fungus10_train_images_features

In [33]:
X_train.shape

(100, 4096)

Weight matrix would have:


*   rows = no. of features
*   columns = no. of classes

In [40]:
np.random.seed(5660)
_,m = fungus10_train_images_features.shape
n = len(set(fungus10_train_images_labels))
W = np.random.uniform(low=-1, high=1, size=(m, n))
b = np.random.uniform(low=-1, high=1, size=(1, n))

In [41]:
W.shape

(4096, 10)

Initial set of weights

In [42]:
print(W[:2])

[[ 0.44140127  0.4690655  -0.73259887 -0.05906308 -0.53820052  0.8147641
  -0.29529803 -0.04197371 -0.93145758  0.22377421]
 [ 0.92699897  0.97547102  0.64151833 -0.70370343 -0.62292231 -0.03680328
   0.6366819   0.89381421  0.26542717  0.27011698]]


In [43]:
print(b)

[[ 0.56534233  0.93943885  0.91182578 -0.06911842 -0.09695704  0.69054724
  -0.8612357  -0.32716141 -0.48437092 -0.82968288]]


We multiply X (100,4096) with W(4096,10) and add b


In [111]:
U, singular_values, V = np.linalg.svd(W,full_matrices=False)

In [112]:
U.shape

(4096, 10)

In [113]:
singular_values.shape

(10,)

In [114]:
V.shape

(10, 10)

In [109]:
def trace_norm(matrix):
    # Compute the singular value decomposition (SVD)
    U, singular_values, Vh = np.linalg.svd(matrix,full_matrices=False)

    # Compute the trace norm as the sum of singular values
    trace_norm_value = np.sum(singular_values)

    return trace_norm_value, U, Vh

In [110]:
trace_norm(W)

(369.0587225211235,
 array([[-0.00831505, -0.00791636,  0.01554518, ...,  0.02511323,
          0.02438435, -0.00978903],
        [ 0.02702271, -0.03117371, -0.0023663 , ...,  0.02158996,
          0.00100944,  0.01075031],
        [ 0.00793354,  0.02256807,  0.00276934, ...,  0.00359363,
         -0.01292101,  0.00381469],
        ...,
        [-0.01166667,  0.00027402,  0.00970164, ...,  0.00852501,
         -0.02850632,  0.01133839],
        [ 0.00760792,  0.02417367,  0.00298942, ...,  0.01093301,
         -0.01570748,  0.01854379],
        [ 0.00262163, -0.0123304 , -0.00707363, ...,  0.02302851,
          0.01409366, -0.0019847 ]]),
 array([[-0.06624644, -0.30462039,  0.33724594, -0.36608441, -0.46158238,
         -0.13328142,  0.39911928,  0.49093928, -0.10774025, -0.11098397],
        [-0.18838267, -0.47468404, -0.15659926,  0.42324781, -0.21111395,
         -0.5697397 , -0.36175656, -0.00626457, -0.18261944, -0.04577824],
        [ 0.28683724,  0.11513005, -0.01636647,  0.3282

In [81]:
def net_input(X, W, b):
    return (X.dot(W) + b)

net_in = net_input(X_train, W, b)
print(f'net input shape:\n {net_in.shape}')
print(f'net input:\n {net_in[:2]}')

net input shape:
 (100, 10)
net input:
 [[ 0.6954171   0.77256549  0.18707491 -0.08701527  0.17142897  0.9085162
  -0.27257828 -0.1387958   0.57632682 -0.92504999]
 [ 0.30649441  0.82180241  0.71058519  0.39058976 -0.93494618  0.8043387
   0.41005151 -0.31011337  0.16902247 -1.34038849]]


In [82]:
def softmax(z):
    return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

smax = softmax(net_in)
print(f'softmax:\n {smax[:2]}')

softmax:
 [[0.14559767 0.15727494 0.08757582 0.0665807  0.08621628 0.18017817
  0.05530436 0.06322085 0.12925108 0.02880013]
 [0.10093817 0.16898605 0.15119931 0.10979374 0.02916786 0.16606054
  0.11195145 0.05448363 0.08797355 0.01944569]]


To get class labels from Probabilities

In [47]:
def to_classlabel(z):
    return z.argmax(axis=1)

print(f'predicted class labels:  {to_classlabel(smax)}')

predicted class labels:  [5 1 2 1 1 4 2 2 0 1 1 2 2 2 5 0 0 1 0 0 1 2 1 2 5 2 2 5 1 5 0 0 5 1 1 5 1
 1 0 1 2 5 1 1 2 2 1 1 5 0 1 1 1 5 5 1 2 5 1 1 2 1 0 1 1 1 1 2 2 5 5 5 2 1
 2 1 4 1 1 5 1 2 5 5 2 3 1 8 1 2 5 0 5 1 1 5 1 2 1 5]


In [136]:
def logistic_loss(X,W,b,y_true):
  prob_scores = softmax(net_input(X, W, b))
  class_ = np.argmax(y_true,axis=1)
  print(class_[:3])
  print(prob_scores[:3])
  numerator=np.exp(prob_scores[np.arange(prob_scores.shape[0]), class_])
  print(numerator[:3])
  exp_arr = np.exp(prob_scores)
  # Sum along the columns
  denominator = np.sum(exp_arr, axis=1)
  print(denominator[:3])
  return -np.log(numerator/denominator)

In [143]:
-np.log(1.1567307/11.06421201)

2.258118092337486

In [137]:
logistic_loss(X=X_train,W=W,b=b, y_true=y_train_true)

[0 0 0]
[[0.14559767 0.15727494 0.08757582 0.0665807  0.08621628 0.18017817
  0.05530436 0.06322085 0.12925108 0.02880013]
 [0.10093817 0.16898605 0.15119931 0.10979374 0.02916786 0.16606054
  0.11195145 0.05448363 0.08797355 0.01944569]
 [0.16501321 0.10825541 0.31217767 0.04838716 0.10451748 0.17874175
  0.01937003 0.01594746 0.03426979 0.01332004]]
[1.1567307  1.10620824 1.1794087 ]
[11.06421201 11.06587916 11.09950916]


array([2.25811809, 2.30292825, 2.24188767, 2.25546786, 2.27337698,
       2.2537862 , 2.27784808, 2.33787005, 2.21473731, 2.26940773,
       2.10931841, 2.24261581, 2.22779101, 2.1876584 , 2.1955556 ,
       2.30187499, 2.25377058, 2.07600046, 2.2653692 , 2.21552681,
       2.29628449, 2.13150489, 2.32256296, 2.1400521 , 2.31832672,
       2.20052384, 1.94860998, 2.30356075, 2.17171822, 2.23453273,
       2.38977056, 2.31662683, 2.36434443, 2.32640448, 2.30766442,
       2.3313598 , 2.28966666, 2.32053657, 2.23610642, 2.36002961,
       2.39025754, 2.36503353, 2.35184703, 2.36237398, 2.33963582,
       2.34623991, 2.35442028, 2.35618442, 2.28769293, 2.3674473 ,
       2.35384477, 2.33888315, 2.26548385, 2.19395904, 2.22674067,
       2.26721999, 2.14350828, 2.11731159, 2.22056804, 2.34533955,
       2.35143615, 2.35878726, 2.38783251, 2.38201617, 2.38204172,
       2.35238012, 2.40665371, 2.37012097, 2.36953712, 2.39598201,
       2.37204759, 2.28234983, 2.38479341, 2.34245392, 2.37897

In [96]:
def regularization_penalty(W, lambda_=10**-2):
  trace_norm_,_,_ = trace_norm(W)
  return lambda_*(trace_norm_**2)

In [97]:
def loss(X, W, b, y_true, lambda_=10**-2):
  regularizer = regularization_penalty(W)
  return np.mean([logistic_loss(X=X_train[i],W=W,b=b, y_true=y_train_true[i])+regularizer for i in range(len(X))])

In [98]:
loss(X_train, W, b, y_train_true)

1364.3574367860135

In [120]:
def gradient_regularizer(W, lambda_=10**-2):
  trace_norm_, U, Vh = trace_norm(W)
  gradient = np.matmul(U,Vh)
  gradient = 2*lambda_*trace_norm_*(gradient)
  return gradient

In [122]:
gradient_regularizer(W).shape

(4096, 10)

In [173]:
class SubgradientDescent():

    """Softmax regression classifier.

    Parameters
    ------------
    eta : float (default: 0.01)
        Learning rate (between 0.0 and 1.0)
    epochs : int (default: 50)
        Passes over the training dataset.
        Prior to each epoch, the dataset is shuffled
        if `minibatches > 1` to prevent cycles in stochastic gradient descent.
    minibatches : int (default: 1)
        The number of minibatches for gradient-based optimization.
        If 1: Gradient Descent learning
        If len(y): Stochastic Gradient Descent (SGD) online learning
        If 1 < minibatches < len(y): SGD Minibatch learning
    random_seed : int (default: None)
        Set random state for shuffling and initializing the weights.

    Attributes
    -----------
    w_ : 2d-array, shape={n_features, 1}
      Model weights after fitting.
    b_ : 1d-array, shape={1,}
      Bias unit after fitting.
    cost_ : list
        List of floats, the average log loss for each epoch.

    """
    def __init__(self,
                 eta=0.01,
                 epochs=50,
                 lambda_=10**-2,
                 minibatches=1,
                 n_classes=10,
                 random_seed=5660):
        self.eta = eta
        self.epochs = epochs
        self.lambda_ = lambda_
        self.minibatches = minibatches
        self.n_classes = n_classes
        self.random_seed = random_seed
        self._encoder = None

    def _init_params(self, weights_shape, bias_shape=(1,), dtype='float64',
                     scale=0.01, random_seed=5660):
        """Initialize weight coefficients."""
        np.random.seed(random_seed)
        w = np.random.normal(loc=0.0, scale=scale, size=weights_shape)
        b = np.zeros(shape=bias_shape)
        return b.astype(dtype), w.astype(dtype)

    def _fit(self, X, y, init_params=True):
        if init_params:
            self._n_features = X.shape[1]

            self.b_, self.w_ = self._init_params(
                weights_shape=(self._n_features, self.n_classes),
                bias_shape=(self.n_classes,),
                random_seed=self.random_seed)
            self.cost_ = []

        y_enc = self._one_hot(y=y)

        for i in range(self.epochs):
            print(f'Epoch#:{i}')
            for idx in self._yield_minibatches_idx(
                    n_batches=self.minibatches,
                    data_ary=y,
                    shuffle=True):
                # givens:
                # w_ -> n_feat x n_classes
                # b_  -> n_classes

                # net_input, softmax and diff -> n_samples x n_classes:
                logistic_loss_gradient = self._logistic_loss_gradient(X[idx],y_enc[idx])
                print(f"logistic_loss_gradient shape = {logistic_loss_gradient.shape}")
                mse = -1.0*np.mean(logistic_loss_gradient, axis=0)
                print(f'mse:{mse}')

                # gradient -> n_features x n_classes
                regularizer_gradient = self._regularizer_gradient()
                print(f'regularizer_gradient shape:{regularizer_gradient.shape}')
                grad = logistic_loss_gradient + regularizer_gradient

                # update in opp. direction of the cost gradient
                self.w_ -= (self.eta * grad +
                            self.eta * self.lambda_ * self.w_)
                self.b_ -= (self.eta * np.sum(-1*logistic_loss_gradient, axis=0))

            # compute cost of the whole epoch
            cost = self._cost(X, y_enc)
            print(f'Epcoh Cost:{cost}')
            self.cost_.append(cost)
        return self

    def fit(self, X, y, init_params=True):
        """Learn model from training data.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.
        init_params : bool (default: True)
            Re-initializes model parametersprior to fitting.
            Set False to continue training with weights from
            a previous model fitting.

        Returns
        -------
        self : object

        """
        if self.random_seed is not None:
            np.random.seed(self.random_seed)
        self._fit(X=X, y=y, init_params=init_params)
        self._is_fitted = True
        return self

    def _predict(self, X):
        probas = self.predict_proba(X)
        return self._to_classlabels(probas)

    def predict(self, X):
        """Predict targets from X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        target_values : array-like, shape = [n_samples]
          Predicted target values.

        """
        if not self._is_fitted:
            raise AttributeError('Model is not fitted, yet.')
        return self._predict(X)

    def predict_proba(self, X):
        """Predict class probabilities of X from the net input.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        Class probabilties : array-like, shape= [n_samples, n_classes]

        """
        net = self._net_input(X, self.w_, self.b_)
        softm = self._softmax(net)
        return softm

    def _net_input(self, X, W, b):
        return (X.dot(W) + b)

    def _softmax(self, z):
        return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

    def _trace_norm(self):
      # Compute the singular value decomposition (SVD)
      U, singular_values, Vh = np.linalg.svd(self.w_,full_matrices=False)

      # Compute the trace norm as the sum of singular values
      trace_norm_value = np.sum(singular_values)

      return trace_norm_value, U, Vh

    def _regularizer_gradient(self):
      trace_norm_, U, Vh = self._trace_norm()
      gradient = np.matmul(U,Vh)
      gradient = 2*self.lambda_*trace_norm_*(gradient)
      return gradient

    def _logistic_loss_gradient(self, X, y_true):
      net = self._net_input(X, self.w_, self.b_)
      softm = self._softmax(net)
      diff = softm - y_true
      gradient = -1*diff
      return gradient

    def logistic_loss(self, X, y_true):
      prob_scores = softmax(net_input(X, self.w_, self.b_))
      class_ = np.argmax(y_true,axis=1)
      numerator=np.exp(prob_scores[np.arange(prob_scores.shape[0]), class_])
      exp_arr = np.exp(prob_scores)
      # Sum along the columns
      denominator = np.sum(exp_arr, axis=1)
      return -np.log(numerator/denominator)

    def regularization_penalty(self, lambda_=10**-2):
      trace_norm_,_,_ = self._trace_norm()
      return lambda_*(trace_norm_**2)

    def _cost(self, X, y_true, lambda_=10**-2):
      regularizer_penalty_ = self.regularization_penalty()
      logistic_loss_ = self.logistic_loss(X, y_true)
      cost_ = logistic_loss_+regularizer_penalty_
      return np.mean(cost_)


    def _to_classlabels(self, z):
        return z.argmax(axis=1)

    def _one_hot(self, y):
      # define one hot encoding
      self._encoder = OneHotEncoder(sparse_output=False)
      # transform data
      return encoder.fit_transform(y.reshape(len(y),1))


    def _yield_minibatches_idx(self, n_batches, data_ary, shuffle=True):
            indices = np.arange(data_ary.shape[0])

            if shuffle:
                indices = np.random.permutation(indices)
            if n_batches > 1:
                remainder = data_ary.shape[0] % n_batches

                if remainder:
                    minis = np.array_split(indices[:-remainder], n_batches)
                    minis[-1] = np.concatenate((minis[-1],
                                                indices[-remainder:]),
                                               axis=0)
                else:
                    minis = np.array_split(indices, n_batches)

            else:
                minis = (indices,)

            for idx_batch in minis:
                yield idx_batch

In [174]:
lr = SubgradientDescent(eta=0.01, epochs=10, minibatches=1, random_seed=5660, )
lr.fit(X_train, fungus10_train_images_labels)

Epoch#:0
logistic_loss_gradient shape = (100, 10)
mse:[-2.87833665e-04 -5.66878791e-05  2.07955529e-04 -3.56364984e-04
 -1.15820267e-04 -3.08870312e-04  1.91899360e-04  2.14790122e-04
  2.50959423e-04  2.59972673e-04]
regularizer_gradient shape:(4096, 10)


ValueError: operands could not be broadcast together with shapes (100,10) (4096,10) 